In [ ]:
#试试读word能不能识别格式。好像有点问题。其他有人实现这方面的功能也是修改docx库的内容去读xml的文件。这个先不管了。
import docx
from docx.oxml import parse_xml
from docx.oxml.xmlchemy import serialize_for_reading
from docx.oxml.ns import nsmap, qn

# 读取docx文件
doc = docx.Document('./Part3/P3_3.docx')

with open('./doc.txt', 'w+') as f:
    
    for p in doc.paragraphs:
        if p.text.replace(' ', '') == '':
            continue
        print("段落内容：", p.text)
        print("段落样式名称：", p.style.name)
        print('常规手段字体名称：', p.style.font.name)
        f.write("段落内容：" + p.text + '\n')
        f.write("段落样式名称：" + p.style.name + '\n')
        f.write('常规手段字体名称：' + p.style.font.name + '\n')
        p_rpr = p.style.element.xpath('w:rPr')[0]
        # print(p_rpr[0].xml)
        if p_rpr.xpath('w:rFonts'):
            try:
            # 一般字体在w:ascii中，中文样式的字体可能在w:eastAsia
            # 找不到的话print一下p_rpr[0].xml找找关于字体信息在哪
                print("段落字体：", p_rpr.xpath('w:rFonts')[0].attrib[qn("w:eastAsia")])
            except:
                print("段落字体：", p_rpr.xpath('w:rFonts')[0].attrib[qn("w:ascii")])



In [33]:
#数据清理。
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from spellchecker import SpellChecker

# nltk.download('wordnet')

def stem_text(tokens):
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    return stemmed_tokens
 
def lemmatize_text(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return lemmatized_tokens


In [2]:

import re, os

table_pattern = r'(# Table of Contents|# Contents)'
figure_pattern = r'(Figures\s{1}\d|Figure|# Fig\.\s{1}\d|Fig\.\s{1}\d\.\d|Fig\.\s{1}\d)[^.?\n]*[.?\n]?'

def get_pages(text):
    pages = text.split('\n---\n')
    return pages

def general_process(text):
    lines_text = text.split('\n')
    processed_lines = []

    for line in lines_text:
        if not re.search(r'^[^#-|].*[^|.?!\-]$', line) and line != '':
            processed_lines.append(line)

    return '\n'.join(processed_lines)

def get_content_table(check_pattern, pages_text, number_of_pages):

    matched_pages = []
    for n, text in enumerate(pages_text):
        match = re.search(check_pattern, text, re.IGNORECASE)
        if match:
            content_table_start_page = n
            matched_pages.append(n+1)
    if not matched_pages:
        print('Cannot match any table of contents!')
        return

    if len(matched_pages) != 1:
        str_matched_pages = map(lambda x: str(x), matched_pages)
        print(f"The recognized table of contents is not unique! The matched pages are {', '.join(str_matched_pages)}. Please manually check it.")
        # content_table_start_page = matched_pages[0] - 1
        return

    table_of_contents = '\n'.join(pages_text[content_table_start_page:content_table_start_page+number_of_pages])
    return table_of_contents

def analysis_table(table_of_contents):
    table_of_contents.replace('|', '')

def clean_figures(figure_pattern, text):
    # 匹配从 "Figures\s{1}\d" 开始到换行符之间的所有内容

    cleaned_text = re.sub(figure_pattern, '', text, flags=re.IGNORECASE)
    return cleaned_text

def remove_cover_page(pages_text, number_of_cover_page, number_of_end_page):
    table_of_contents = '\n'.join(pages_text[number_of_cover_page-1:number_of_end_page-1:])
    return table_of_contents

clean_file = './part3_txt/P3_10_cropped.txt'
output_path = os.path.join('./part3_clean', clean_file.split('/')[-1].replace('cropped', 'cleaned'))

print(output_path)
# if not os.path.exists(output_path):
#     os.mkdir(output_path)

with open(clean_file, 'r') as f:
    text = f.read()
    text = clean_figures(figure_pattern, text)
    pages = get_pages(text)
    print(len(pages))
    table = remove_cover_page(pages, 29, 627)
    # table = get_content_table(table_pattern, pages, 2)
    with open(output_path, 'w') as w:
        w.write(table)

./part3_clean\P3_10_cleaned.txt
640
